# Azure Cosmos DB コレクションへのバッチ取り込み

このノートで、あなたは、

1. IoTDeviceInfo データセットを ADLS Gen2 からデータフレームに読み込みます
2. Azure Cosmos DB コレクションにデータフレームを書き込みます

>**ご存知ですか？** [Azure Cosmos DB 用 Azure Synapse Link](https://docs.microsoft.com/ja-jp/azure/cosmos-db/synapse-link) は、Azure Cosmos DB の運用データに対してほぼリアルタイムの分析を実行できるハイブリッドトランザクション分析処理 (HTAP) 機能です。
&nbsp;

>**ご存知ですか？** [Azure Cosmos DB 分析ストア](https://docs.microsoft.com/ja-jp/azure/cosmos-db/analytical-store-introduction)は、トランザクションワークロードに影響を与えることなく、Azure Cosmos DB の運用データに対して大規模な分析を可能にする完全に分離された列ストアです。
&nbsp;


### 1. IoTDeviceInfo データセットを ADLS Gen2 からデータフレームに読み込む
>**ご存知ですか？** Synapse ワークスペースは ADLS Gen2 ストレージアカウントにアタッチされており、デフォルトのストレージアカウントに配置されたファイルには、以下のように相対パスを使用してアクセスできます。
&nbsp;



In [3]:
dfDeviceInfo = (spark
                .read
                .csv("/IoTData/IoTDeviceInfo.csv", header=True, inferSchema='true')
              )

### 2. Azure Cosmos DB コレクションにデータフレームを書き込む
>**ご存知ですか？** 「cosmos.oltp」は、Cosmos DB トランザクションストアへの接続を可能にする Spark フォーマットです。

>**ご存知ですか？** Cosmos DB コレクションへの取り込みは、分析ストアが有効になっているかどうかに関係なく、常にトランザクションストアを通じて実行されます。

In [4]:
dfDeviceInfo.write\
            .format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "CosmosDBIoTDemo")\
            .option("spark.cosmos.container", "IoTDeviceInfo")\
            .option("spark.cosmos.write.upsertEnabled", "true")\
            .mode('append')\
            .save()
